**$T_1.$** Soit $X_1$, ..., $X_n$ des variables aléatoires iid de loi $\mathcal{N}(\theta, 1)$, où $\theta$ est inconnu.
1.  (a) Montrer que $\widehat{\theta_n} = \frac{1}{n} \sum\limits_{k=1}^{n}X_k$ converge presque sûrement vers $\theta$.
    
    (b) Déterminer la loi de $\sqrt{n}(\widehat{\theta_n} - \theta)$.
1. Soit $\alpha$ $\in$ $]0, \;1[$ fixé et $\Phi$ la fonction répartition de la loi Gaussienne $\mathcal{N}(\theta, 1)$. Déterminer en fonction de $\alpha$, de $n$, de $\Phi^{-1}$ et de $\theta_0$ la valeur optimale de $K_\alpha$ pour que la relation $\mathbb{P}_{\theta = \theta_0}(T_K=1) \leq \alpha$ soit satisfaite avec une égalité.


//TODO T1 1.a, 1.b, 2

**$S_1.$** Créer un code python permettant:
1. de simuler $n$ variables gaussiennes indépendantes de loi $\mathcal{N}(\theta_0, 1)$,
1. de calculer pour $\alpha$ et $\theta_0$ fixés, la constante $K_\alpha$ et qui vous renvoie la valeur optimale de votre règle
de décision.
1. Pour $\theta_0 = 3$, $\alpha = 0.05$ et $n = 100$ répéter $N = 100$ fois l’expérience et compter le nombre fois où
vous avez décidé $H_0$. On appelle ce nombre $N_n$ quelle est sa loi?
1. Pour $n \in \{100,\; 1000\}$ et $N \in \{100, 1000, 5000\}$, Simuler $500$ valeurs de la variable aléatoire $N_n$ et
les représenter sous forme d’histogramme. Que remarquez-vous?

In [7]:
# Imports
import math

import numpy as np
from scipy.special import erfinv

In [ ]:
#S1.1
def n_gauss_iid(theta0: float, n: int) -> np.ndarray:
    """Generates n independent gaussian random variables of standard deviation 1 and average theta0.

    Args:
        theta0 (float): average value
        n (int): number of random variables to generate.
    Returns:
        np.ndarray: array with sample of n independent gaussian random variables.
    """
    return np.random.normal(theta0, 1, n)

In [ ]:
#S1.2
def phi_inv(p: float) -> float:
    """Quantile function (inverse of the cumulative function) of the standard normal distribution.

    Args:
        p (float): probability (p in [0, 1])

    Returns:
        float: x which phi(x) = p
    """
    return math.sqrt(2.)*erfinv(2.*p - 1.)

def get_k_alpha(theta0: float, alpha: float, n: int) -> float:
    """_summary_

    Args:
        theta0 (float): _description_
        alpha (float): _description_
        n (int): _description_

    Returns:
        float: optimal value K alpha
    """
    # TODO
    pass